In [ ]:
# import re
# import nltk
# from nltk.tokenize import word_tokenize
# %pip install spacy
# import spacy
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# %pip install transformers
# from transformers import AutoTokenizer, AutoModel
# import torch

: 

In [3]:
# Reinstall NumPy
%pip uninstall -y numpy
%pip install numpy

# Verify NumPy installation
import numpy as np
print(np.__version__)

# Install pandas
%pip install pandas

# Install Matplotlib
%pip install numpy pandas matplotlib



Found existing installation: numpy 2.2.0
Uninstalling numpy-2.2.0:
  Successfully uninstalled numpy-2.2.0
Note: you may need to restart the kernel to use updated packages.
  Using cached numpy-2.2.0-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
Using cached numpy-2.2.0-cp312-cp312-macosx_14_0_arm64.whl (5.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires h5py>=3.11.0, which is not installed.
tensorflow 2.18.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, which is not installed.
scikit-learn 1.6.0 requires threadpoolctl>=3.1.0, which is not installed.
tensorboard 2.18.0 requires markdown>=2.6.8, which is not installed.
tensorboard 2.18.0 requires protobuf!=4.24.0,>=3.19.6, which is not installed.
tensorboard 2.18.0 requires werkzeug>=1.0.1, which is not installed.
keras 3.7.0 requir

In [27]:
# Import necessary libraries
import json
import pandas as pd
import matplotlib.pyplot as plt
import re

def read_test_cases(file_path):
    """
    Read test cases from a JSON file.

    Args:
        file_path (str): The path to the JSON file.

    Returns:
        list: A list of test cases.
    """
    with open(file_path, 'r') as file:
        contents = file.read()
        return contents
    
def preprocess_text(path):
    """Preprocess text data."""
    text = read_test_cases(path)
    return text

def clean_text(text):
    """Remove unwanted characters and symbols from text."""
    pattern = r'(?<="train\.SRC": ").+(?=", "train\.EXR")'
    train_src = re.finditer(pattern, text)
    train_src_arr = []
    for match in train_src:
        train_src_arr.append(match.group())
    return train_src_arr  

def preprocess_train_top(text):
    """Get the training topics from the text."""
    pattern = r'(?<="train\.TOP": ").+(?=", "train\.TOP-DECOUPLED")'
    train_top = re.finditer(pattern, text)
    train_top_arr = []
    for match in train_top:
        train_top_arr.append(match.group())
    return train_top_arr

def preprocess_train_top_decoupled(text):
    """Get the training topics from the text."""
    pattern = r'(?<="train\.TOP-DECOUPLED": ").+(?="})'
    train_top_decoupled = re.finditer(pattern, text)
    train_top_decoupled_arr = []
    for match in train_top_decoupled:
        train_top_decoupled_arr.append(match.group())
    return train_top_decoupled_arr



all_data = preprocess_text('test.json')
print(all_data)

train_src = clean_text(all_data)
print(train_src)
print('-------------------')
train_top = preprocess_train_top(all_data)
print(train_top)
print('-------------------')
train_top_decoupled = preprocess_train_top_decoupled(all_data)
print(train_top_decoupled)



[
    {"train.SRC": "i'd like a pizza with carrots barbecue pulled pork and cheeseburger without thin crust", "train.EXR": "(ORDER (PIZZAORDER (NUMBER 1 ) (TOPPING CARROTS ) (TOPPING BBQ_PULLED_PORK ) (TOPPING CHEESEBURGER ) (NOT (STYLE THIN_CRUST ) ) ) )", "train.TOP": "(ORDER i'd like (PIZZAORDER (NUMBER a ) pizza with (TOPPING carrots ) (TOPPING barbecue pulled pork ) and (TOPPING cheeseburger ) without (NOT (STYLE thin crust ) ) ) )", "train.TOP-DECOUPLED": "(ORDER (PIZZAORDER (NUMBER a ) (TOPPING carrots ) (TOPPING barbecue pulled pork ) (TOPPING cheeseburger ) (NOT (STYLE thin crust ) ) ) )"},
    {"train.SRC": "i'd like a pizza with banana pepper grilled chicken and white onions without thin crust", "train.EXR": "(ORDER (PIZZAORDER (NUMBER 1 ) (TOPPING BANANA_PEPPERS ) (TOPPING GRILLED_CHICKEN ) (TOPPING WHITE_ONIONS ) (NOT (STYLE THIN_CRUST ) ) ) )", "train.TOP": "(ORDER i'd like (PIZZAORDER (NUMBER a ) pizza with (TOPPING banana pepper ) (TOPPING grilled chicken ) and (TOPPING

In [30]:
def parse_tree_to_hierarchy(tree_str):
    """
    Parse a structured tree string into a hierarchical format (like the visual example).

    Args:
        tree_str (str): The input parse tree string.

    Returns:
        dict: A hierarchical representation of the parse tree.
    """
    def recursive_parse(tokens):
        tree = {}
        while tokens:
            token = tokens.pop(0)
            if token == ")":  # End of subtree
                return tree
            elif token.startswith("("):  # Start of a new subtree
                node = token[1:]  # Remove the '('
                tree[node] = []  # Initialize node's children
                child = recursive_parse(tokens)
                if child:
                    tree[node].append(child)
            else:
                tree[token] = None  # Leaf node (no children)
        return tree

    # Tokenize the tree string
    tokens = tree_str.replace("(", " (").replace(")", " )").split()
    return recursive_parse(tokens)

def display_tree(tree, level=0):
    """
    Display the hierarchical tree in an indented format.

    Args:
        tree (dict): The hierarchical tree.
        level (int): The current indentation level.
    """
    for key, value in tree.items():
        print(" " * (level * 4) + f"{key}")
        if isinstance(value, list) and value:
            for child in value:
                display_tree(child, level + 1)
        elif value is None:  # Leaf node
            pass


# Convert and Display the Hierarchical Parse Tree
for i, tree_str in enumerate(train_top_decoupled):
    print(f"Data {i + 1}:")
    print(f"train_src: {train_src[i]}") # Display the training source

    print(f"Tree {i + 1}:")
    tree = parse_tree_to_hierarchy(tree_str) # Parse the tree string
    display_tree(tree)
    print(tree)
    print("-" * 30)




Data 1:
train_src: i'd like a pizza with carrots barbecue pulled pork and cheeseburger without thin crust
Tree 1:
ORDER
    PIZZAORDER
        NUMBER
            a
        TOPPING
            cheeseburger
        NOT
            STYLE
                thin
                crust
{'ORDER': [{'PIZZAORDER': [{'NUMBER': [{'a': None}], 'TOPPING': [{'cheeseburger': None}], 'NOT': [{'STYLE': [{'thin': None, 'crust': None}]}]}]}]}
------------------------------
Data 2:
train_src: i'd like a pizza with banana pepper grilled chicken and white onions without thin crust
Tree 2:
ORDER
    PIZZAORDER
        NUMBER
            a
        TOPPING
            white
            onions
        NOT
            STYLE
                thin
                crust
{'ORDER': [{'PIZZAORDER': [{'NUMBER': [{'a': None}], 'TOPPING': [{'white': None, 'onions': None}], 'NOT': [{'STYLE': [{'thin': None, 'crust': None}]}]}]}]}
------------------------------
Data 3:
train_src: i want one regular pizza without any fried onio

In [2]:
# import spacy
# from spacy.cli import download

# # Download the Spacy model
# download("en_core_web_sm")

In [3]:
# import nltk
# import os

# # Define a specific path for nltk data
# nltk_data_path = os.path.expanduser("~/nltk_data")
# os.makedirs(nltk_data_path, exist_ok=True)

# # Force download punkt
# nltk.download('punkt', download_dir=nltk_data_path)

# # Add the directory to nltk's data path
# if nltk_data_path not in nltk.data.path:
#     nltk.data.path.append(nltk_data_path)

# # Debugging: Print the data path
# print("NLTK Data Path:", nltk.data.path)


In [4]:
# from nltk.tokenize import word_tokenize

# sample_text = "I would like a large chicago style pizza."
# tokens = word_tokenize(sample_text)
# print("Tokens:", tokens)


In [ ]:

# Import necessary libraries
import nltk
import spacy
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from transformers import AutoTokenizer, AutoModel
import torch

# Download NLTK Tokenizer Resources
nltk.download('punkt')

# Load Spacy Model for Lemmatization
nlp = spacy.load("en_core_web_sm")

# Your existing code

In [29]:
# ######################################
# # Step 1: Data Cleaning
# ######################################
# def clean_text(text):
#     """
#     Clean input text by removing special characters, extra spaces, and standardizing case.
#     """
#     text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove special characters
#     text = text.lower()  # Convert to lowercase
#     text = text.replace("dont", "do not").replace("id", "i would")  # Expand contractions
#     text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
#     return text

In [30]:
# ######################################
# # Step 2: Tokenization
# ######################################
# def tokenize_text(text):
#     """
#     Tokenize text into words.
#     """
#     return word_tokenize(text)

In [31]:
# ######################################
# # Step 3: Lemmatization (Optional)
# ######################################
# def lemmatize_text(text):
#     """
#     Perform lemmatization on the text to reduce words to their base forms.
#     """
#     doc = nlp(text)
#     return " ".join([token.lemma_ for token in doc])

In [32]:
# ######################################
# # Step 4: Entity Annotation Placeholder (Manual or Automated)
# ######################################
# # Assume pre-labeled dataset is provided for supervised training.
# # Add your annotation logic or preprocessing steps for labels here.

# def annotate_entities(example_text):
#     """
#     Placeholder for entity annotation.
#     Add custom logic or use tools like Prodigy/Label Studio.
#     """
#     # Example Output: Pre-labeled tokens with BIO format
#     tokens = tokenize_text(example_text)
#     tags = ["O"] * len(tokens)  # Replace with actual annotation logic
#     return list(zip(tokens, tags))

In [33]:
# ######################################
# # Step 5: Convert Data to Model-Friendly Format
# ######################################
# def prepare_bio_format(data):
#     """
#     Convert text and labels into BIO-tagged format for training.
#     """
#     bio_data = []
#     for text in data:
#         annotated = annotate_entities(text)
#         bio_data.extend(annotated)
#     return bio_data

In [34]:
# ######################################
# # Step 6: Dataset Splitting
# ######################################
# def split_dataset(data):
#     """
#     Split dataset into training, validation, and test sets.
#     """
#     train, temp = train_test_split(data, test_size=0.2, random_state=42)
#     val, test = train_test_split(temp, test_size=0.5, random_state=42)
#     return train, val, test

In [35]:
# ######################################
# # Step 7: Feature Extraction
# ######################################
# # Bag of Words (BoW)
# def extract_bow_features(texts):
#     vectorizer = CountVectorizer()
#     return vectorizer.fit_transform(texts)

# # TF-IDF
# def extract_tfidf_features(texts):
#     vectorizer = TfidfVectorizer()
#     return vectorizer.fit_transform(texts)

# # Contextual Word Embeddings
# def extract_contextual_embeddings(texts):
#     tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
#     model = AutoModel.from_pretrained("bert-base-uncased")
#     embeddings = []
#     for text in texts:
#         inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
#         outputs = model(**inputs)
#         embeddings.append(outputs.last_hidden_state.mean(dim=1).detach().numpy())
#     return torch.tensor(embeddings)

In [ ]:
# ######################################
# # Main Workflow Example
# ######################################
# import nltk
# nltk.download('punkt')

# if __name__ == "__main__":
#     # Example Dataset
#     dataset = [
#         "I would like a large chicago style pizza with extra cheese and no onions.",
#         "Two medium cans of diet pepsi and one large bottle of coke.",
#     ]

#     # Clean Text
#     cleaned_data = [clean_text(text) for text in dataset]

#     # Tokenize Text
#     tokenized_data = [tokenize_text(text) for text in cleaned_data]

#     # Optional Lemmatization
#     lemmatized_data = [lemmatize_text(text) for text in cleaned_data]

#     # Prepare Data for BIO Tagging
#     bio_data = prepare_bio_format(cleaned_data)

#     # Split Dataset
#     train, val, test = split_dataset(cleaned_data)

#     # Extract Features
#     bow_features = extract_bow_features(train)
#     tfidf_features = extract_tfidf_features(train)
#     contextual_features = extract_contextual_embeddings(train)

#     print("Preprocessing completed!")